# With `exactextract`

Rasterix includes dask-aware wrappers for `exactextract.exact_extract`'s coverage estimation.

## Read data

### Read in some raster data

In [ ]:
import xarray as xr

ds = xr.tutorial.open_dataset("eraint_uvz")
ds = ds.rio.write_crs("epsg:4326")
ds

### Read in example geometries

In [ ]:
import geodatasets
import geopandas as gpd

world = gpd.read_file(geodatasets.get_path("naturalearth land"))
world

## Calculate coverage

In [ ]:
from rasterix.rasterize.exact import coverage

n = coverage(ds, world[["geometry"]], xdim="longitude", ydim="latitude")
n

In [ ]:
n.isel(geometry=112).plot()

Optionally clip to the `total_bounds` of the geometries passed in

In [ ]:
from rasterix.rasterize.exact import coverage

coverage(
    ds,
    world[["geometry"]].iloc[slice(112, 113)],
    xdim="longitude",
    ydim="latitude",
    clip=True,
).plot()

Notice that the output data is a `sparse.COO` array with a new dimension `geometry`. The input geometries are propagated as a coordinate variable named `geometry`.

Different coverage weights are supported. Extra attributes `units` and `long_name` are assigned as appropriate.

In [ ]:
from rasterix.rasterize.exact import coverage

n = coverage(ds, world[["geometry"]], xdim="longitude", ydim="latitude", coverage_weight="fraction")
n

## Out-of-core support

As with other rasterization code, various combinations of chunked and in-memory arrays and geometries are supported.

In [ ]:
chunked = ds.chunk({"latitude": -1, "longitude": 120})
d = coverage(chunked, world[["geometry"]], xdim="longitude", ydim="latitude")
d

In [ ]:
xr.testing.assert_identical(d, n)

In [ ]:
import dask_geopandas as dgpd

dd = coverage(
    ds.chunk({"latitude": -1, "longitude": 240}),
    dgpd.from_geopandas(world[["geometry"]], npartitions=3),
    xdim="longitude",
    ydim="latitude",
)
dd

In [ ]:
xr.testing.assert_identical(dd, n)